In [39]:
import spglib as spg
import ase
from ase.visualize import view

import ase.spacegroup as aspg
from ase.visualize import view
from ase.io import read,write
import numpy as np
import pandas as pd
from collections import Iterable
import cPickle as pck
import spglib as spg
import sys, os
from make_input.utils import rotation_matrix,isCellSkewed,unskewCell,get_symprec,get_relative_angle
from make_input.qe_input import makeCard,makeNamelist,SG2ibrav,SG2wyckoff,makeQEInput_sg,makeQEInput_ibrav0,makeQEInput
from make_input.custom_frame import frame2qe_format,get_ibrav2_frame
from make_input.qe_run import run_qe_local
from make_input.raw_info import SG2BravaisLattice
from read_output.parser import extract_floats,finishedProperly,fn2info,fn2num,get_filenames,get_patterns,get_energy_per_atom


# import info and input structures

In [3]:
fileNames = {}
infoPath = './info/'
structurePath = './structures/'
fileNames['crystals'] = structurePath + 'partial_input_crystals_sg3-230.pck'
fileNames['wyck'] = infoPath+'SpaceGroup-multiplicity-wickoff-info.pck'
fileNames['general info'] = infoPath+'SpaceGroup-general-info.pck'
fileNames['elements info'] = infoPath+'General-Info-Elements-fast.pck'

In [4]:
with open(fileNames['crystals'],'rb') as f:
    crystals = pck.load(f)
with open(fileNames['wyck'],'rb') as f:
    WyckTable = pck.load(f)
SGTable = pd.read_pickle(fileNames['general info'])
ElemTable = pd.read_pickle(fileNames['elements info'])

# examples

In [5]:
from make_input.qe_input import frame2change,ibrav0,NOPROBLEM,dont_print_wyck,tricky_sg

## single sg

In [22]:
sg,it = 210,0
cc = crystals[sg][it]
dirName = './test_run/check_input_errors_ibrav3/sg_{}-f_{}'.format(sg,it)


In [23]:
print cc.get_scaled_positions()[0]

[ 0.30321532  0.3032168   0.58934866]


In [24]:
input_str = makeQEInput(cc,sg,WyckTable,SGTable,ElemTable,
                 zatom = 14,rhocutoff = 20 * 4,wfccutoff = 20,
                 calculation_type='"scf"',smearing=1e-2,
                pressure=0,press_conv_thr=0.5,cell_factor=2,
                etot_conv_thr=1e-4,forc_conv_thr=1e-3,nstep=150,
                scf_conv_thr=1e-1,print_forces=True,print_stress=False,
                restart=False,
                 kpt = [2,2,2],Nkpt=None,kpt_offset = [0,0,0],ppPath='"../pseudo/"')
# print inpt

In [ ]:
exitstatus = run_qe_local(input_str,dirName,verbose=False,
                 path2mpi='/usr/bin/',np=4,path2pw='/home/musil/source/qe-6.1/bin/')

## several sg

In [19]:
zatom = 14
calculation_type = '"scf"'
sgs = [202]

kpt = [2,2,2]
Nkpt = None
# rhocutoff ,wfccutoff = None,None
rhocutoff ,wfccutoff = 20*4,20
smearing = 1e-2
etot_conv_thr = 1e-4
forc_conv_thr = 1e-3
nstep = 150
scf_conv_thr = 1e-1
ppPath='"./pseudo/SSSP_acc_PBE/"'

dataPath = './test_run/check_input_errors_ibrav3/'
dirNames = {(sg,it):dataPath + 'sg_{}-f_{}'.format(sg,it)
            for sg in  sgs for it in range(len(crystals[sg]))}

In [20]:
for (sg,it),dirName in dirNames.iteritems():
    print dirName
    crystal = crystals[sg][it]
    input_str = makeQEInput(crystal,sg,WyckTable,SGTable,ElemTable,
                    zatom = zatom,rhocutoff = rhocutoff,wfccutoff = wfccutoff,
                    calculation_type=calculation_type,smearing=smearing,
                    pressure=0,press_conv_thr=0.5,cell_factor=2,
                    etot_conv_thr=1e-4,forc_conv_thr=1e-3,nstep=150,
                    scf_conv_thr=scf_conv_thr,print_forces=False,
                    kpt = kpt,Nkpt=Nkpt ,kpt_offset = [0,0,0],
                    ppPath=ppPath)

    print 'sending the calc'
    exitstatus = run_qe_local(input_str,dirName,verbose=False,
                 path2mpi='/usr/bin/',np=4,path2pw='/home/musil/source/qe-6.1/bin/')

    print exitstatus

./test_run/check_input_errors_ibrav3/sg_202-f_0
sending the calc
(0, 'No error')


# ibrav 3 test

In [38]:
sg,it = 220,0
cc = crystals[sg][it]
dirName = './test_run/check_input_errors_ibrav3/sg_{}-f_{}'.format(sg,it)

In [29]:
print cc.get_scaled_positions()[0]

[ 0.62501026  0.76843214  0.48157344]


In [33]:
fns = get_filenames('./test_run/check_input_errors_ibrav3/', fn_pattern='qe.out', dir_pattern=None)
fns

['./test_run/check_input_errors_ibrav3/sg_220-f_0-0/qe.out',
 './test_run/check_input_errors_ibrav3/sg_202-f_0-1/qe.out']

In [45]:
fn = './test_run/check_input_errors_ibrav3/sg_220-f_0-0/qe.out'
oo = read(fn)
view(oo)

In [37]:
print oo.get_cell()

[[ 4.22005118  4.22005118  4.22005118]
 [-4.22005118  4.22005118  4.22005118]
 [-4.22005118 -4.22005118  4.22005118]]


In [44]:
dd,_,_ = get_ibrav2_frame(cc,220)
view(dd)

# ibrav 9 test

In [54]:
dataPath = './test_run/check_input_errors_ibrav9/'

In [55]:
sg,it = 40,0
cc = crystals[sg][it]
dirName = dataPath+'sg_{}-f_{}/'.format(sg,it)

In [60]:
view(cc)

In [56]:
print cc.get_cell_lengths_and_angles()
print cc.get_scaled_positions()[0]

[  8.10960373   3.90162918   2.25168312  90.          90.          90.        ]
[ 0.61485391  0.83332859  0.49      ]


In [57]:
fns = get_filenames(dataPath, fn_pattern='qe.out', dir_pattern=None)
fns

['./test_run/check_input_errors_ibrav9/sg_40-f_0-0/qe.out']

In [59]:
fn = './test_run/check_input_errors_ibrav9/sg_40-f_0-0/qe.out'
oo = read(fn)
view(oo)

In [37]:
print oo.get_cell()

[[ 4.22005118  4.22005118  4.22005118]
 [-4.22005118  4.22005118  4.22005118]
 [-4.22005118 -4.22005118  4.22005118]]


In [44]:
dd,_,_ = get_ibrav2_frame(cc,220)
view(dd)